# Ligrec Benchmark
This notebook benchmarks `gr.ligrec` for *squidpy* and *rapids-singlecell*.

To run this notebook please make sure you have a working rapids enviroment with all nessaray dependencies. Run the data_downloader notebook first to create the AnnData object we are working with. In this example workflow we'll be looking at a dataset of ca. 90000 cells from [Quin et al., Cell Research 2020](https://www.nature.com/articles/s41422-020-0355-0).

In [1]:
import scanpy as sc
import squidpy as sq
import cupy as cp
import rapids_singlecell as rsc
from rapids_singlecell.cunnData import cunnData

import warnings
warnings.filterwarnings("ignore")

In [2]:
import rmm
rmm.reinitialize(
    managed_memory=False, # Allows oversubscription
    pool_allocator=False, # default is False
    devices=0, # GPU device IDs to register. By default registers only GPU 0.
)
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

## Load and Prepare Data

We load the sparse count matrix from an `h5ad` file using Scanpy. The sparse count matrix will then be placed on the GPU and run basic preprocessing for `rsc.gr.ligrec`

In [3]:
%%time
adata = sc.read("h5/adata.raw.h5ad")

CPU times: user 2.01 s, sys: 188 ms, total: 2.19 s
Wall time: 2.2 s


In [4]:
%%time
cudata = rsc.cunnData.cunnData(adata=adata)

CPU times: user 1.26 s, sys: 762 ms, total: 2.02 s
Wall time: 3.13 s


In [5]:
adata.shape

(93575, 33694)

In [6]:
cudata.shape

(93575, 33694)

In [7]:
%%time
rsc.pp.flag_gene_family(cudata,gene_family_name="MT", gene_family_prefix="MT-")

CPU times: user 4.88 ms, sys: 0 ns, total: 4.88 ms
Wall time: 4.81 ms


In [8]:
%%time
rsc.pp.calculate_qc_metrics(cudata,qc_vars=["MT"])

CPU times: user 75.4 ms, sys: 4.12 ms, total: 79.6 ms
Wall time: 120 ms


In [9]:
%%time
cudata = cudata[cudata.obs["n_genes_by_counts"] < 5000]
cudata.shape

CPU times: user 89.6 ms, sys: 57.5 ms, total: 147 ms
Wall time: 305 ms


(92666, 33694)

In [10]:
%%time
cudata = cudata[cudata.obs["pct_counts_MT"] < 20]
cudata.shape

CPU times: user 3.27 ms, sys: 19.7 ms, total: 23 ms
Wall time: 22.4 ms


(91180, 33694)

In [11]:
%%time
rsc.pp.filter_genes(cudata,min_count=3)

filtered out 8034 genes based on n_cells_by_counts
CPU times: user 59.8 ms, sys: 26.3 ms, total: 86.1 ms
Wall time: 86 ms


In [12]:
%%time
rsc.pp.normalize_total(cudata,target_sum=1e4)

CPU times: user 0 ns, sys: 1.62 ms, total: 1.62 ms
Wall time: 1.26 ms


In [13]:
%%time
rsc.pp.log1p(cudata)

CPU times: user 6.52 ms, sys: 1.02 ms, total: 7.55 ms
Wall time: 6.63 ms


In [14]:
%%time
adata = cudata.to_AnnData()
adata.raw = adata

CPU times: user 112 ms, sys: 44.2 ms, total: 157 ms
Wall time: 156 ms


In [15]:
adata

AnnData object with n_obs × n_vars = 91180 × 25660
    obs: 'nGene', 'nUMI', 'CellFromTumor', 'PatientNumber', 'TumorType', 'TumorSite', 'CellType', 'n_genes_by_counts', 'total_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'total_counts_MT', 'pct_counts_MT', 'log1p_total_counts_MT'
    var: 'gene_ids', 'MT', 'n_cells_by_counts', 'total_counts', 'mean_counts', 'pct_dropout_by_counts', 'log1p_total_counts', 'log1p_mean_counts'
    uns: 'log1p'

## Ligrec Benchmark

First we download the interactions so that both function get evaluated in the same way

In [16]:
interactions = rsc.squidpy_gpu._ligrec._get_interactions()

Next, we execute the function using both the *rapids-singlecell* and *squidpy* versions for comparison

In [17]:
%%time
res_rsc = rsc.gr.ligrec(
    adata,
    n_perms=1000,
    interactions=interactions,
    cluster_key="CellType",
    copy=True,
    use_raw=True,
)

CPU times: user 3.32 s, sys: 298 ms, total: 3.62 s
Wall time: 3.68 s


In [18]:
res_rsc["pvalues"]

cluster_1     Alveolar                                               \
cluster_2     Alveolar B_cell Cancer     EC Epithelial Erythroblast   
source target                                                         
EPOR   TRPC3       NaN    NaN    NaN    NaN        NaN          NaN   
       JAK2      0.504  0.943  0.971  0.000      0.163          NaN   
FYN    JAK2      1.000  1.000  1.000  1.000      1.000          NaN   
CCL2   JAK2      0.000  0.000  0.000  0.000      0.000          NaN   
KIT    JAK2        NaN    NaN    NaN    NaN        NaN          NaN   
...                ...    ...    ...    ...        ...          ...   
MANSC1 GPR55       NaN    NaN    NaN    NaN        NaN          NaN   
MSMB   GPR55       NaN    NaN    NaN    NaN        NaN          NaN   
COPA   P2RY6     0.990  1.000  0.000  0.966        NaN          NaN   
VSTM1  ADGRG3      NaN    NaN    NaN    NaN        NaN          NaN   
TYMP   HPGDS       NaN  1.000    NaN    NaN        NaN          NaN   

cluster_1                                          ...   T_cell                \
cluster_2     Fibroblast Mast_cell Myeloid T_cell  ... Alveolar B_cell Cancer   
source target                                      ...                          
EPOR   TRPC3         NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
       JAK2          0.0     0.461   0.000  0.061  ...      1.0    1.0    1.0   
FYN    JAK2          1.0     1.000   1.000  1.000  ...      0.0    0.0    0.0   
CCL2   JAK2          0.0     0.000   0.000  0.000  ...      1.0    1.0    1.0   
KIT    JAK2          NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
...                  ...       ...     ...    ...  ...      ...    ...    ...   
MANSC1 GPR55         NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
MSMB   GPR55         NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
COPA   P2RY6         1.0       NaN   0.227    NaN  ...      1.0    1.0    1.0   
VSTM1  ADGRG3        NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
TYMP   HPGDS         NaN     0.000   1.000    NaN  ...      NaN    1.0    NaN   

cluster_1                                                                  \
cluster_2         EC Epithelial Erythroblast Fibroblast Mast_cell Myeloid   
source target                                                               
EPOR   TRPC3     NaN        NaN          NaN        NaN       NaN     NaN   
       JAK2    0.103      0.566          NaN      0.014     0.979     0.0   
FYN    JAK2    0.000      0.000          NaN      0.000     0.000     0.0   
CCL2   JAK2    1.000      1.000          NaN      1.000     1.000     1.0   
KIT    JAK2      NaN        NaN          NaN        NaN       NaN     NaN   
...              ...        ...          ...        ...       ...     ...   
MANSC1 GPR55     NaN        NaN          NaN        NaN       NaN     NaN   
MSMB   GPR55     NaN        NaN          NaN        NaN       NaN     NaN   
COPA   P2RY6   1.000        NaN          NaN      1.000       NaN     1.0   
VSTM1  ADGRG3    NaN        NaN          NaN        NaN       NaN     NaN   
TYMP   HPGDS     NaN        NaN          NaN        NaN     0.000     1.0   

cluster_1             
cluster_2     T_cell  
source target         
EPOR   TRPC3     NaN  
       JAK2      1.0  
FYN    JAK2      0.0  
CCL2   JAK2      1.0  
KIT    JAK2      NaN  
...              ...  
MANSC1 GPR55     NaN  
MSMB   GPR55     NaN  
COPA   P2RY6     NaN  
VSTM1  ADGRG3    NaN  
TYMP   HPGDS     NaN  

[7873 rows x 100 columns]

In [19]:
%%time
res_sq = sq.gr.ligrec(
    adata,
    n_perms=1000,
    interactions=interactions,
    cluster_key="CellType",
    copy=True,
    use_raw=True,
    n_jobs= 32,
)

  0%|          | 0/1000 [00:00<?, ?permutation/s]

CPU times: user 20.3 s, sys: 1.88 s, total: 22.2 s
Wall time: 52.1 s


In [20]:
res_sq["pvalues"]

cluster_1     Alveolar                                               \
cluster_2     Alveolar B_cell Cancer     EC Epithelial Erythroblast   
source target                                                         
EPOR   TRPC3       NaN    NaN    NaN    NaN        NaN          NaN   
       JAK2      0.496  0.930  0.968  0.000      0.168          NaN   
FYN    JAK2      1.000  1.000  1.000  1.000      1.000          NaN   
CCL2   JAK2      0.000  0.000  0.000  0.000      0.000          NaN   
KIT    JAK2        NaN    NaN    NaN    NaN        NaN          NaN   
...                ...    ...    ...    ...        ...          ...   
MANSC1 GPR55       NaN    NaN    NaN    NaN        NaN          NaN   
MSMB   GPR55       NaN    NaN    NaN    NaN        NaN          NaN   
COPA   P2RY6     0.983  0.999  0.000  0.959        NaN          NaN   
VSTM1  ADGRG3      NaN    NaN    NaN    NaN        NaN          NaN   
TYMP   HPGDS       NaN  1.000    NaN    NaN        NaN          NaN   

cluster_1                                          ...   T_cell                \
cluster_2     Fibroblast Mast_cell Myeloid T_cell  ... Alveolar B_cell Cancer   
source target                                      ...                          
EPOR   TRPC3         NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
       JAK2        0.000     0.462   0.000  0.077  ...      1.0    1.0    1.0   
FYN    JAK2        1.000     1.000   1.000  1.000  ...      0.0    0.0    0.0   
CCL2   JAK2        0.000     0.000   0.000  0.000  ...      1.0    1.0    1.0   
KIT    JAK2          NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
...                  ...       ...     ...    ...  ...      ...    ...    ...   
MANSC1 GPR55         NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
MSMB   GPR55         NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
COPA   P2RY6       0.997       NaN   0.222    NaN  ...      1.0    1.0    1.0   
VSTM1  ADGRG3        NaN       NaN     NaN    NaN  ...      NaN    NaN    NaN   
TYMP   HPGDS         NaN     0.000   1.000    NaN  ...      NaN    1.0    NaN   

cluster_1                                                                  \
cluster_2         EC Epithelial Erythroblast Fibroblast Mast_cell Myeloid   
source target                                                               
EPOR   TRPC3     NaN        NaN          NaN        NaN       NaN     NaN   
       JAK2    0.124      0.548          NaN      0.012     0.992     0.0   
FYN    JAK2    0.000      0.000          NaN      0.000     0.000     0.0   
CCL2   JAK2    1.000      1.000          NaN      1.000     1.000     1.0   
KIT    JAK2      NaN        NaN          NaN        NaN       NaN     NaN   
...              ...        ...          ...        ...       ...     ...   
MANSC1 GPR55     NaN        NaN          NaN        NaN       NaN     NaN   
MSMB   GPR55     NaN        NaN          NaN        NaN       NaN     NaN   
COPA   P2RY6   1.000        NaN          NaN      1.000       NaN     1.0   
VSTM1  ADGRG3    NaN        NaN          NaN        NaN       NaN     NaN   
TYMP   HPGDS     NaN        NaN          NaN        NaN     0.000     1.0   

cluster_1             
cluster_2     T_cell  
source target         
EPOR   TRPC3     NaN  
       JAK2      1.0  
FYN    JAK2      0.0  
CCL2   JAK2      1.0  
KIT    JAK2      NaN  
...              ...  
MANSC1 GPR55     NaN  
MSMB   GPR55     NaN  
COPA   P2RY6     NaN  
VSTM1  ADGRG3    NaN  
TYMP   HPGDS     NaN  

[7873 rows x 100 columns]